In [2]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.dates as mdates
import statistics

In [8]:
market = Market()
speculation_db = ADatabase("speculation")

In [4]:
market.connect()
prices = market.retrieve("pdr_prices")
market.disconnect()
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [24]:
value = True
training_year = 4
positions = 5
start_date = datetime(2016,1,1)
end_date = datetime(2022,7,1)

In [25]:
speculation_db.connect()
simulation = speculation_db.retrieve(f"pdr_sim")
simulation["year"] = [int(x) for x in simulation["year"]]
sim = prices.copy().merge(simulation[(simulation["training_year"]==training_year)],on=["year","week","ticker"],how="left")
sim = sim.dropna()
sim["prediction"] = (sim["xgb_prediction"] + sim["skl_prediction"] + sim["cat_prediction"]) / 3
sim["delta"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]
speculation_db.disconnect()

In [26]:
included_cols = [
                "date","sell_date","ticker"
                 ,"adjclose","sell_price"
                 ,"delta","projected_delta"
                 ,"position","req","value","training_year"
                 ,"signal","swap"
                ]

In [27]:
def trade_factory(trade,date,current_price,current_gain,position,req,value,training_year,signal,swap):
    trade["sell_date"] = date
    trade["sell_price"] = min(trade["adjclose"]*(1+req),current_price)
    trade["delta"] = (trade["sell_price"] - trade["adjclose"]) / trade["adjclose"]
    trade["position"] = position
    trade["req"] = req
    trade["value"] = value
    trade["training_year"] = training_year
    trade["signal"] = signal
    trade["swap"] = swap
    return trade

In [28]:
def purchase_factory(date,position_dictionary,offering):
    purchase_dictionary = {}
    purchase_dictionary["adjclose"] = offering["adjclose"]
    purchase_dictionary["ticker"] = offering["ticker"]
    purchase_dictionary["amount"] = int(position_dictionary["cash"] / offering["adjclose"])
    purchase_dictionary["date"] = date
    purchase_dictionary["projected_delta"] = offering["delta"]
    return purchase_dictionary

In [29]:
def portfolio_init(start_date,positions,req,value,signal):
    portfolio = {"date":start_date}
    portfolio["positions"] = {}
    portfolio["req"] = req
    portfolio["value"] = value
    portfolio["signal"] = signal
    for position in range(positions):
        portfolio["positions"][position] = {"cash":100/positions,"asset":{}}
    return portfolio

In [30]:
def current_ticker_list(portfolio,positions):
    current_tickers = [portfolio["positions"][position]["asset"]["ticker"] for position in range(positions)
                                  if len(portfolio["positions"][position]["asset"].keys()) > 0]
    return current_tickers

In [33]:
signals = [0.1,0.3,0.5]
reqs = [0.05,0.1,0.15]
values = [True]
speculation_db.connect()
for signal in tqdm(signals):
    for value in values:
        iterration_sim = sim.copy()
        if value==False:
            iterration_sim["delta"] = iterration_sim["delta"] * -1
        for req in reqs:
            portfolio = portfolio_init(start_date,positions,req,value,signal)
            date = start_date
            current_tickers = []
            while date < end_date:
                year = date.year
                day_jump = 3 if date.weekday() == 4 else 1
                for position in range(positions):
                    current_tickers = current_ticker_list(portfolio,positions)
                    position_dictionary = portfolio["positions"][position]
                    try:
                        ## check recs
                        todays_recs = iterration_sim[(iterration_sim["date"]==date) & (iterration_sim["delta"] > signal) & (~iterration_sim["ticker"].isin(current_tickers))]
                        todays_recs.sort_values("delta",ascending=False,inplace=True)
                        if (len(position_dictionary["asset"].keys()) == 0) and (todays_recs.index.size > position) and (date.weekday() <= 2) and (prices[prices["date"]==date].index.size > 0):
                            offering = todays_recs.iloc[position]
                            purchase_dictionary = purchase_factory(date,position_dictionary,offering)
                            position_dictionary["asset"] = purchase_dictionary
                            position_dictionary["cash"] = position_dictionary["cash"] - offering["adjclose"] * purchase_dictionary["amount"]
                            current_tickers = current_ticker_list(portfolio,positions)
                        # check exits
                        if len(position_dictionary["asset"].keys()) > 0:
                            asset_dictionary = position_dictionary["asset"]
                            days_passed = int((date - asset_dictionary["date"]).days)
                            max_days = 7 + asset_dictionary["date"].weekday() + 1
                            ticker = asset_dictionary["ticker"]
                            price_data = prices[(prices["ticker"]==ticker) & (prices["date"]==date)]
                            if price_data.index.size > 0:
                                current_price = price_data.iloc[0]["adjclose"].item()
                                buy_price = asset_dictionary["adjclose"]
                                current_gain = (current_price - buy_price) / buy_price
                                if current_gain > req or days_passed >= max_days:
                                    trade = trade_factory(asset_dictionary,date,current_price,current_gain,position
                                                          ,req,value,training_year,signal,swap=False)
                                    trade_df = pd.DataFrame([trade])
                                    speculation_db.store("oc_trades",trade_df[included_cols])
                                    position_dictionary["cash"] = position_dictionary["cash"] + current_price * trade["amount"]
                                    position_dictionary["asset"] = {}
                                    current_tickers = current_ticker_list(portfolio,positions)
                                else:
                                    offerings =  iterration_sim[(iterration_sim["date"]==date) & (iterration_sim["delta"] >= signal) & (~iterration_sim["ticker"].isin(current_tickers))]
                                    offerings.sort_values("delta",ascending=False,inplace=True)
#                                     print("swap",offerings.index.size,position)
                                    if offerings.index.size > position:
                                        offering = offerings.iloc[position]
                                        if (current_gain > 0) and (offering["delta"] > (asset_dictionary["projected_delta"] - current_gain)) and (offering["delta"] >= signal):
                                            trade = trade_factory(asset_dictionary,date,current_price,current_gain,position
                                                              ,req,value,training_year,signal,swap=True)                       
                                            trade_df = pd.DataFrame([trade])
                                            speculation_db.store("oc_trades",trade_df[included_cols])
                                            position_dictionary["cash"] = position_dictionary["cash"] + current_price * trade["amount"]
                                            purchase_dictionary = purchase_factory(date,position_dictionary,offering)
                                            position_dictionary["asset"] = purchase_dictionary
                                            position_dictionary["cash"] = position_dictionary["cash"] - offering["adjclose"] * purchase_dictionary["amount"]
                                            current_tickers = current_ticker_list(portfolio,positions)
                    except Exception as e:
                        print(str(e))
                    date = date+timedelta(days=day_jump)
speculation_db.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [15:45<00:00, 315.15s/it]


In [ ]:
(4-1)//3 + 1